In [520]:
import numpy as np
import itertools
import random
import simpy
import csv

In [521]:
# All units are in minutes

WEEKS = 4

# Simulation time in minutes
SIM_TIME = 168 * WEEKS * 60              

# number of docks/berth
DOCKS = 2

# number of tugs 
TUGS = 100

# number of cranes 
CRANES = 4


# shape parameter of Erlang
K = 3

UNLOAD_SPEED = 1
UNLOADING_TIME = 120.0

# average inter-arrival times for ships of type 1 to 3
AVG_TRAVEL_TYPE1_3 = 180.0

# average unloading time for ships of type 1
UNLOAD_TIME_TYPE_1 = UNLOADING_TIME

# average unloading time for ships of type 2
UNLOAD_TIME_TYPE_2 = UNLOADING_TIME

# average unloading time for ships of type 3
UNLOAD_TIME_TYPE_3 = UNLOADING_TIME

BERTHING_TIME = 90
DEBERTHING_TIME = 30.0


"""----- Ship distribution by type ----"""
# ship distribution of ships type 1 to 3 {type 1: 25%, type 2: 55%, type 3: 20%}
SHIP_DISTR = [1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,3,3,3,3]

"""-----------LOG----------------"""
# wait tug to berthing
tug_wt1 = []
# wait for berth
berth_wt = []
# wait tug to deberthing
tug_wt2 = []
# waiting for crane
crane_wt = []

# help to analyze results
ship_departure_times = []

In [522]:
def ship_generator(env, tug, dock, crane):
    #Generate new ships.
    i = 0
    while True:
        yield env.timeout(1)
        # randomly select a ship type from SHIP_DISTR
        shiptype = random.choice(SHIP_DISTR)
        env.process(ship('Ship %d' % i, env, tug, dock, crane, shiptype))
        yield env.timeout(time_to_arrive())
        i += 1

def time_to_arrive():
    # Erlang/Gamma distribution with shape k, scale theta
    theta = AVG_TRAVEL_TYPE1_3/K
    return random.gammavariate(K,theta)
    
def time_to_berth():
    return random.expovariate(1.0/BERTHING_TIME)

def time_to_deberth():
    return random.expovariate(1.0/DEBERTHING_TIME)

def time_to_unload(shiptype):
    unloadTime = {
            1 : UNLOAD_TIME_TYPE_1,
            2 : UNLOAD_TIME_TYPE_2,
            3 : UNLOAD_TIME_TYPE_3,
                  }
    return random.expovariate(1.0/unloadTime[shiptype])

def ship(name, env, tug, dock, crane, shiptype):
    print('%s arriving at port at %.1f' % (name, env.now))
    with tug.request() as req:
        # Request the tug
        wt = env.now
        yield req
        wt = env.now - wt
        tug_wt1.append(wt)
        
    # request for dock and process ship at dock
    env.process(shipAtDock(name, env, tug, dock, crane, shiptype))

        
def tug_transport(name, env, tug, shiptype):
    """Arrives at the port or the dock after a 1h delay and transports the ship."""
    # print('Tug starts %s transport at %.1f' % (name, env.now))
    yield env.timeout(TUG_TIME_ONE_WAY)
    # print('Tug stops %s transport at %.1f' % (name, env.now))


def shipAtDock(name, env, tug, dock, crane, shiptype):
    with dock.request() as req:
        # Request a dock
        wt = env.now
        yield req
        wt = env.now - wt
        berth_wt.append(wt)
        
        # Berthing
        yield env.timeout(time_to_berth())
        
        with crane.request() as req:
            # Request the crane
            wt = env.now
            yield req
            wt = env.now - wt
            crane_wt.append(wt)
            
            # Loading/unloading
            print('%s of shiptype %d unloading at %.1f' % (name, shiptype, env.now))           
            yield env.timeout(time_to_unload(shiptype))
            print('%s unloaded at %.1f' % (name, env.now))
        
        with tug.request() as req:
            # Request the tug
            wt = env.now
            yield req
            wt = env.now - wt
            tug_wt2.append(wt)

        # Deberthing
        yield env.timeout(time_to_deberth())
            
    ship_departure_times.append(env.now)
    print('%s departed at %.1f' % (name, env.now))

In [523]:
# Setup and start the simulation
print('Marine Transport Simulation')

# Create environment and start processes
env = simpy.Environment()

# ships_type_4 = simpy.Resource(env, 1)

# Resource with capacity of usage slots that can be requested by processes.
tug = simpy.Resource(env, TUGS)

# Resource with capacity of usage slots that can be requested by processes.
dock = simpy.Resource(env, DOCKS)

# Resource with capacity of usage slots that can be requested by processes.
crane = simpy.Resource(env, CRANES)

# Process an event yielding generator.
env.process(ship_generator(env, tug, dock, crane))

# Execute!
env.run(until=SIM_TIME)

Marine Transport Simulation
Ship 0 arriving at port at 1.0
Ship 0 of shiptype 3 unloading at 87.4
Ship 1 arriving at port at 150.0
Ship 2 arriving at port at 273.0
Ship 0 unloaded at 278.2
Ship 0 departed at 283.4
Ship 3 arriving at port at 319.6
Ship 1 of shiptype 2 unloading at 364.1
Ship 1 unloaded at 370.5
Ship 2 of shiptype 3 unloading at 385.3
Ship 2 unloaded at 442.0
Ship 2 departed at 447.9
Ship 3 of shiptype 1 unloading at 473.0
Ship 3 unloaded at 476.3
Ship 4 arriving at port at 482.9
Ship 1 departed at 485.1
Ship 3 departed at 499.0
Ship 4 of shiptype 3 unloading at 657.3
Ship 5 arriving at port at 691.1
Ship 4 unloaded at 711.4
Ship 4 departed at 735.8
Ship 5 of shiptype 2 unloading at 787.3
Ship 6 arriving at port at 845.5
Ship 5 unloaded at 898.9
Ship 5 departed at 947.5
Ship 7 arriving at port at 964.8
Ship 6 of shiptype 3 unloading at 966.8
Ship 6 unloaded at 970.3
Ship 6 departed at 983.6
Ship 7 of shiptype 1 unloading at 1002.6
Ship 7 unloaded at 1038.8
Ship 7 departe

In [524]:
num_ship = min(len(tug_wt1), len(tug_wt2), len(berth_wt))
ship_departure_times = ship_departure_times[0:num_ship-1]
ship_delay_times = np.add(berth_wt[0:num_ship-1], np.add(tug_wt1[0:num_ship-1], tug_wt2[0:num_ship-1]))

In [525]:
print ship_delay_times

[  0.           0.          10.3462511  128.30914491   2.21840256
   0.           0.           0.           0.           0.
  30.07640463   0.           0.           0.          36.48563244
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.          44.71948501   0.
  45.78885059   0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.          32.72382105  82.86493837   0.           0.
   0.           0.           0.           0.           0.
   0.          15.19332413 174.68117698 128.3810897  199.78618089
 142.34460356   0.           0.           0.          19.75519297
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.          49.93479648  86.31782814 172.60299958
  45.7689905  227.8450323  219.1

In [463]:
# Initialize the file and write first result
with open('simpy_results3.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerow(["Time", "Delay"])
    for i in range(0,num_ship-1):
        writer.writerow([ship_departure_times[i], ship_delay_times[i]])

In [526]:
# Append to the file if it already exists
with open('simpy_results3.csv', 'a') as f:
    writer = csv.writer(f)
    for i in range(0,num_ship-1):
        writer.writerow([ship_departure_times[i], ship_delay_times[i]])